This notebook illustrates Fisher analysis for **$\Lambda$CDM** cosmology with a simulator of **Stage IV 3x2pt photometric probes**

That is, our observables are the weak-lensing, galaxy clustering and the cross-correlated angular power spectra at different tomographic redshift bins

We perform Fisher analysis in order to get an initial estimate of the 1$\sigma$ errors of the 1d posteriors. We will use this to have a good guess for the prior boundaries: $[\theta^0_i-5\sigma_i, \theta^0_i+5\sigma_i]$. The main steps are:

- Define the simulator, which uses routines defined in class_interface.py (it requires having _CLASS_ installed)

- Compute and store the inverse of the Fisher matrix 

In [1]:
# General imports
import swyft #the version used is 0.4.6.
import torch

from class_interface import Main

import numpy   as np
from time      import time
from scipy     import stats

True

Set fiducial values for ($H_0$, 100$\omega_b$, $\omega_c$, $n_s$, $\mathrm{ln}(10^{10}A_s)$, $A_{\rm IA}$, $\eta_{\rm IA}$, $b_1$, $b_2$, $b_3$, $b_4$, $b_5$, $b_6$, $b_7$, $b_8$, $b_9$, $b_{10}$)

In [3]:
N_pars = 17 # total number of parameters (cosmo+nuisance)
fiducial = np.array([67.,2.2445,0.1206,0.96,3.0568,1.72,-0.41,1.09977,1.22025,1.2724,1.31662,1.35812,1.39982,1.44465,1.4965,1.56525,1.74299])

beta_IA = 0.0 #we neglect dependence of Intrinsic Alignment on the galaxy luminosity function 

### Define the simulator class

In [5]:
Nbin_z = 10  # Number of tomographic z-bins
Nbin_ell = 30  # Number of ell-bin edges
N_spectra = Nbin_z*(2*Nbin_z+1) # total number of spectra (210 for 10 z-bins)

lmin = 10
lmax = 2000

class Simulator(swyft.Simulator):
    def __init__(self, zmin = 0.01, kmax = 10., bounds = None):

        super().__init__()
        self.transform_samples = swyft.to_numpy32
        
        # These are the specifications of the experiment
        self.specs = {'fsky': 0.35,
                      'gal_per_arcmin': 30.,
                      'sigma_eps': 0.3,
                      'Nbin_ell': Nbin_ell,
                      'lmin': lmin,
                      'lmax': lmax,
                      'zmin': zmin,
                      'kmax': kmax,           
                      'use_obs': {'GC': True, 'WL': True},
                      'feedback': False, # We don't want to print stuff
                      'ddm':False} # We don't want ddm corrections on the non-linear Pk
        
        # path for n(z) and luminosity files
        self.survey = {'luminosity': './Input_files/luminosity_ratio.dat',
                      'nz': './Input_files/nzTabISTF.dat'
                       }
        
        # Initialize interface with CLASS
        self.calculator = Main(self.specs,self.survey)
        # define default prior region           
        self.sample_z = swyft.RectBoundSampler(stats.uniform(np.array([60,0.8,0.10,0.91,2.8,1.55,-0.45,0.9,1.1,1.15,1.2,1.25,1.3,1.35,1.4,1.4,1.6]), 
                                                             np.array([15,2.7,0.04,0.12,0.5,0.34,0.08,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3])), 
                                                             bounds = bounds) 
   
        #define the keys for the different spectra
        WLcols = ['L{}xL{}'.format(i,j) for i in range(1,Nbin_z+1) for j in range(i,Nbin_z+1)]
        XCcols = ['G{}xL{}'.format(i,j) for i in range(1,Nbin_z+1) for j in range(1,Nbin_z+1)]
        GCcols = ['G{}xG{}'.format(i,j) for i in range(1,Nbin_z+1) for j in range(i,Nbin_z+1)]
        self.all_cols = WLcols+XCcols+GCcols  
        
        # define the ell bins
        ell_lims    = np.logspace(np.log10(lmin),np.log10(lmax),Nbin_ell)
        self.ells   = 0.5*(ell_lims[:-1]+ell_lims[1:])

        # select fiducial parameters
        self.params_fid = {'output':'mPk',
                          'non linear':'halofit',
                          'H0': fiducial[0],
                          'omega_b': fiducial[1]*1e-2,
                          'omega_cdm': fiducial[2],
                          'n_s': fiducial[3],
                          'ln10^{10}A_s': fiducial[4],
                          'tau_reio': 0.05,
                          'N_ur':2.0328,
                          'N_ncdm':1,
                          'm_ncdm': 0.06
                           }
            
        #Update with list of nuisance parameters
        self.params_fid.update({'A_IA': fiducial[5],'eta_IA': fiducial[6],'beta_IA': beta_IA,
                                'b_1': fiducial[7],'b_2': fiducial[8],'b_3': fiducial[9],'b_4': fiducial[10],
                                'b_5': fiducial[11],'b_6': fiducial[12],'b_7': fiducial[13],'b_8': fiducial[14],
                                'b_9': fiducial[15],'b_10':fiducial[16]})
            
        #obtain covariance matrix for fiducial spectra
        self.observables_fid = self.calculator.get_calculations(self.params_fid)
        self.Cls_fid = self.observables_fid.Cls
        self.covmat_fid, self.noisy_Cls_fid = self.calculator.get_noisy_Cls(self.Cls_fid)
        
        # Perform Cholesky decomposition of the covariance matrix
        self.L_fid = {}
        for ind,ell in enumerate(self.ells):
            self.L_fid[str(int(ell))] = torch.linalg.cholesky(torch.tensor(self.covmat_fid[str(int(ell))].values).double())
      
    def get_sample_Cls(self,z):

        #Select cosmo parameters
        self.params = {'output':'mPk',
                      'non linear':'halofit',
                      'H0': z[0],
                      'omega_b': z[1]*1e-2,
                      'omega_cdm': z[2],
                      'n_s': z[3],
                      'ln10^{10}A_s': z[4],
                      'tau_reio': 0.05,
                      'N_ur':2.0328,
                      'N_ncdm':1,
                      'm_ncdm': 0.06
                       }
    
        #Update with list of nuisance parameters
        self.params.update({'A_IA': z[5],'eta_IA': z[6],'beta_IA': beta_IA,
                            'b_1': z[7],'b_2': z[8],'b_3': z[9],'b_4': z[10],'b_5': z[11],
                            'b_6': z[12],'b_7': z[13],'b_8': z[14],'b_9': z[15],'b_10':z[16]})
        
        #compute un-noised C_ells spectra
        self.observables = self.calculator.get_calculations(self.params)
        self.Cls = self.observables.Cls    
        self.Cls.pop('ells')
        #Note: self.Cls is a dictionary, but in the simulator we need to have numpy arrays as the output of the graph.nodes            
        vals_Cls = np.array(list(self.Cls.values))
        return vals_Cls.transpose()
    
    def get_sample_noise(self):
        #generate noise realization (using covariance matrix for fiducial cosmology)
        self.noise = self.calculator.get_realization(None,self.covmat_fid,self.all_cols)
        #Note: self.noise is a dictionary, but in the simulator we need to have numpy arrays as the output of the graph.nodes            
        vals_noise = np.array(list(self.noise.values()))
        return vals_noise
    
    def build(self, graph):
        # define the computational graph of the simulator
        z = graph.node('z', self.sample_z) #draw parameters from the prior
        C_ells = graph.node('C_ells', self.get_sample_Cls, z) #compute un-noised Cls
        noise  = graph.node('noise', self.get_sample_noise) # get the noise, we will add it to the un-noised Cls later

Create an instance of the simulator class

In [ ]:
sim = Simulator()
shapes, dtypes = sim.get_shapes_and_dtypes()
print("shapes:", shapes)
print("dtypes:", dtypes)

### Perform Fisher analysis

In [ ]:
%%time

#First, compute and store numerical derivatives
def finiteD(i, eps=1e-2):
    fiducial_1 = list(fiducial)
    fiducial_1[i] = fiducial[i]*(1+eps)
    fiducial_2 = list(fiducial)
    fiducial_2[i] = fiducial[i]*(1-eps)
    C1 = sim.sample(conditions = {'z': fiducial_1},targets = ['C_ells'])
    C2 = sim.sample(conditions = {'z': fiducial_2},targets = ['C_ells'])
    output = np.zeros((Nbin_ell-1,N_spectra))
    for ell in range(Nbin_ell-1):
        output[ell,:] = [(C1['C_ells'][idx_spec,ell] - C2['C_ells'][idx_spec,ell])/(fiducial[i]*(2*eps)) for idx_spec in range(N_spectra)]
    return output

ders = [finiteD(i) for i in range(N_pars)] 

#Then, compute Fisher matrix
F = np.zeros((N_pars,N_pars))

ell_lims = np.logspace(np.log10(lmin), np.log10(lmax), Nbin_ell)
ells = 0.5*(ell_lims[:-1]+ell_lims[1:])

for i in range(N_pars):
    for j in range(N_pars):
        d1=ders[i]
        d2=ders[j]
        term = 0.
        for ellind,ell in enumerate(ells):
            covmat = sim.covmat_fid[str(int(ell))]
            icovmat = np.linalg.inv(covmat)
            term += np.dot(d1[ellind,:],np.matmul(icovmat,d2[ellind,:]))
        F[i,j] = term

# We store the inverse of Fisher matrix
Finv=np.linalg.inv(F)
np.save('./Fisher/Finv_LCDM.npy',Finv)